In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from scipy import stats
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
tfd = tfp.distributions

/usr/local/lib/python2.7/dist-packages/scipy/optimize/_trlib/__init__.py:1: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._trlib import TRLIBQuadraticSubproblem
/usr/local/lib/python2.7/dist-packages/scipy/optimize/_numdiff.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._group_columns import group_dense, group_sparse
/usr/local/lib/python2.7/dist-packages/scipy/optimize/_minimize.py:37: ImportWarning: Not importing directory '/usr/local/lib/python2.7/dist-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/usr/local/lib/python2.7/dist-packages/scipy/stats/_continuous_distns.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _stats


In [2]:
# verify GPU
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [2]:
def read_image(fn,dims=None):
#  read image into data matrix    
    gdal.AllRegister()
    inDataset = gdal.Open(fn,GA_ReadOnly)
    cols = inDataset.RasterXSize
    rows = inDataset.RasterYSize    
    bands = inDataset.RasterCount
    if dims:
        x0,y0,cols,rows = tuple(dims)
    else:
        x0 = 0
        y0 = 0
    G = np.zeros((rows*cols,bands))
    for b in range(bands):
        band = inDataset.GetRasterBand(b+1)
        G[:,b] = band.ReadAsArray(x0,y0,cols,rows).astype(np.float32).ravel()
    return G

def tf_cov(x,ws):
#  weighted covariance matrix and weighted means of uncentered data tensor x  
    x = tf.transpose(x) # transposed data matrix
    S = tf.shape(x)
    N = S[0]
    sumw = tf.reduce_sum(ws)
    ws = tf.reshape( tf.tile(ws,[N]), S ) 
    xw = tf.multiply(x,ws)
    mx = tf.divide( tf.reduce_sum(xw, axis=1, keepdims=True), sumw )
    mx1 = tf.matmul(mx,tf.transpose(mx))
    xw = tf.multiply(x,tf.sqrt(ws))
    vx = tf.matmul(xw, tf.transpose(xw))/sumw
    return vx-mx1, mx

def geneiv(A,B):
    '''solves A*x = lambda*B*x for tensors A, B 
       returns eigenvectors in columns'''
    Li = tf.linalg.inv(tf.cholesky(B))
    C = tf.matmul(tf.matmul(Li,A),Li,transpose_b=True)
    lambdas,V = tf.linalg.eigh(C)
    return lambdas, tf.matmul(tf.transpose(Li),V)

def mad(x1,x2,pvs):  
    m = tf.shape(x1)[0]
    N = tf.shape(x1)[1]
    x = tf.concat([x1,x2],axis=1)
    itr = 0
    while itr<50:
        itr += 1
    #  weighted covariance and means    
        cov,ms = tf_cov(x,pvs)
        ms1 = tf.transpose(ms[:N]) #row vectors
        ms2 = tf.transpose(ms[N:])
        s11 = cov[:N,:N]
        s12 = cov[:N,N:]
        s21 = cov[N:,:N]
        s22 = cov[N:,N:]
        c1 = tf.matmul(tf.matmul(s12,tf.linalg.inv(s22)),s21)
        b1 = s11
        c2 = tf.matmul(tf.matmul(s21,tf.linalg.inv(s11)),s12)
        b2 = s22
        rho2,A = geneiv(c1,b1)
        _   ,B = geneiv(c2,b2)
        rho = tf.sqrt(rho2[::-1])
        A = A[:,::-1]  
        B = B[:,::-1]
    #  ensure positive correlation between each pair of canonical variates        
        cov = tf.diag_part(tf.matmul(tf.matmul(tf.transpose(A),s12),B))
        cov = tf.diag(tf.divide(cov,tf.abs(cov)))
        B = tf.matmul(B,cov)  
    #  chisqr
        sig2s = 2*(1-rho)
        sig2s = tf.reshape( tf.tile(sig2s,[m]), (m,N) )
        ms1 = tf.reshape( tf.tile(ms1[0],[m]), (m,N) )
        ms2 = tf.reshape( tf.tile(ms2[0],[m]), (m,N) )

        CV1 = tf.matmul( x1-ms1, A )
        CV2 = tf.matmul( x2-ms2, B )
        MADs = CV1 - CV2
        chisqr = tf.reduce_sum(tf.square(MADs)/sig2s, axis=1) 

        N1 = tf.cast(N,dtype=tf.float64)
        one = tf.constant(1.0,dtype=tf.float64)

        pvs = tf.subtract(one,tfd.Chi2(N1).cdf(chisqr))

    return (MADs, chisqr, rho)
    

x1 = tf.placeholder(tf.float64)
x2 = tf.placeholder(tf.float64)
ws = tf.placeholder(tf.float64)
 
mad_op = mad(x1,x2,ws)    

In [3]:
%%time
img1 = read_image('myimagery/LT5_19980329_sub.tif',dims=[0,0,600,600])
img2 = read_image('myimagery/LT5_19980516_sub.tif',dims=[0,0,600,600])
m,N = img1.shape
pvs = np.ones(m)
with tf.Session() as sess:    
    MADs,chisqr,rho = sess.run(mad_op,feed_dict = {x1:img1,x2:img2,ws:pvs})
    print rho


[0.9846993  0.85795521 0.8438159  0.75635176 0.50051521 0.42624577]
CPU times: user 19.8 s, sys: 3.2 s, total: 23 s
Wall time: 8.83 s
